In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

!pip install evaluate
!pip install rouge_score
!pip install bert_score

from evaluate import load

!pip install langdetect
from langdetect import detect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f07c566b617736ddece4384977a8a437eb1a10b193cdc9fbff20170878e249c1
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=0d42b07d4341f035d3ff32b69d91d6a9db858469adf20de8b7a3a6bfc5819699
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
# Load the model and tokenizer from the saved path
#fine_tuned_model_path = "/kaggle/input/lama2_model_q_and_a/pytorch/default/1"
#fine_tuned_model = AutoModelForCausalLM.from_pretrained(model,ignore_mismatched_sizes=True)
#fine_tuned_model_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# base_model
#base_model_path = "NousResearch/Llama-2-7b-chat-hf"
#base_model_tokenizer = AutoTokenizer.from_pretrained(base_model_path)
#base_model = AutoModelForCausalLM.from_pretrained(base_model_path,ignore_mismatched_sizes=True)

In [3]:
from datasets import load_dataset
# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(guanaco_dataset, split = 'train')

README.md:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

(…)-00000-of-00001-9ad84bb9cf65a42f.parquet:   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [4]:
import re
import matplotlib.pyplot as plt
from datasets import load_dataset

def load_model_and_tokenizer(model_path):
    """
    Load a pre-trained model and its tokenizer from the specified path.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path, ignore_mismatched_sizes=True)
    return model, tokenizer

def load_dataset_split(dataset_name, split='train'):
    """
    Load a specific split of a dataset by name.
    """
    return load_dataset(dataset_name, split=split)

def clean_text(response):
    """
    Removes instruction tags and extracts main response text.
    """
    cleaned = re.sub(r"<s>\[INST\].*?\[/INST\]", "", response, flags=re.DOTALL).strip()
    return cleaned


# Path to the fine-tuned model
#fine_tuned_model_path = "/kaggle/input/lama2_model_q_and_a/pytorch/default/1" ##
base_model_path = "NousResearch/Llama-2-7b-chat-hf"

# Load fine-tuned model and tokenizer
fine_tuned_model, fine_tuned_tokenizer = load_model_and_tokenizer(base_model_path)

# Load the dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset_split(guanaco_dataset)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
def extract_and_generate_response(dataset_sample):
    """Extract the instruction text and generate response if text is in English."""
    # Step 1: Extract text between [INST] and [/INST] using regex
    #inst_text = re.findall(r"<s>\[INST\](.*?)\[/INST\]", dataset_sample, flags=re.DOTALL)
    inst_text = dataset_sample
    if inst_text:
        #inst_text = inst_text[0].strip()  # Extract the first match and clean up
        # Step 2: Detect if the text is in English
        if detect(inst_text) == 'en':
            # Step 3: Generate response using the extracted instruction
            response = pipe(f"<s>[INST] {inst_text} [/INST]")
            return response[0]['generated_text']
        else:
            return None  # Skip non-English instructions
    return None  # No instruction text found 
    
def process_dataset(dataset_samples):
    """Process the dataset and generate responses for English instructions only."""
    generated_responses = []
    for sample in dataset_samples:
        generated_response = extract_and_generate_response(sample)
        if generated_response:  # Only append if a response was generated
            generated_responses.append(generated_response)
    return generated_responses
    
def compute_metrics(predictions, references):
    """
    Compute BLEU, ROUGE, and BERTScore metrics for the given predictions and references.
    """
    bertscore = load("bertscore")
    bert_score = bertscore.compute(predictions=predictions, references=references, lang="en")
    
    # Averaging F1 scores across all tokens
    bert_f1_mean = sum(bert_score["f1"]) / len(bert_score["f1"])
    print("\nBERTScore (F1 mean):", bert_f1_mean)

    return bert_f1_mean

In [6]:
# Initialize the model and tokenizer
pipe = pipeline(task="text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, max_length=800)

Device set to use cpu


In [7]:
# Define prompts
# prompts = ["Compose a professional email with the following points:\n\nMe chinese cook 10 years\nGood good cook\nPeople love my food\nCan you hire me?",]

# Initialize lists to store predictions and references
predictions = []
references = []

output_file = "inference_output.txt"
with open(output_file, "w") as f:
    # Generate and print responses for each prompt
    for idx, sample in enumerate(dataset.select(range(30))):
        print(sample['text'])
        instruction_text = sample['text']
        # Step 1: Extract text between [INST] and [/INST] using regex
        inst_text = re.findall(r"<s>\[INST\](.*?)\[/INST\]", instruction_text, flags=re.DOTALL)
    
        if inst_text:
            inst_text = inst_text[0].strip()  # Extract the first match and clean up
        # Step 2: Detect if the text is in English
        model_response = extract_and_generate_response(inst_text)
        if model_response: 
            #print('mod', model_response)
            ground_truth_response = dataset[idx]
            #print(ground_truth_response)
            
            # Clean responses
            ground_truth_response = ground_truth_response['text']
            clean_ground_truth = clean_text(ground_truth_response)
            clean_generated = clean_text(model_response)
    
            # Append to lists
            predictions.append(clean_generated)
            references.append(clean_ground_truth)
            
    
            # Print responses
            print(f"Instruction: {inst_text}")
            print(f"Ground Truth Response: {clean_ground_truth}")
            print(f"Model Response: {clean_generated}\n")

            # Write individual results to file
            f.write(f"Instruction: {inst_text}\n")
            f.write(f"Generated Response: {clean_generated}\n")
            
    bert_f1_mean = compute_metrics(predictions, references)
    f.write(f"BERTScore (F1 mean): {bert_f1_mean}\n\n")

<s>[INST] Me gradué hace poco de la carrera de medicina ¿Me podrías aconsejar para conseguir rápidamente un puesto de trabajo? [/INST] Esto vale tanto para médicos como para cualquier otra profesión tras finalizar los estudios aniversarios y mi consejo sería preguntar a cuántas personas haya conocido mejor. En este caso, mi primera opción sería hablar con otros profesionales médicos, echar currículos en hospitales y cualquier centro de salud. En paralelo, trabajaría por mejorar mi marca personal como médico mediante un blog o formas digitales de comunicación como los vídeos. Y, para mejorar las posibilidades de encontrar trabajo, también participaría en congresos y encuentros para conseguir más contactos. Y, además de todo lo anterior, seguiría estudiando para presentarme a las oposiciones y ejercer la medicina en el sector público de mi país. </s>


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST] Самый великий человек из всех живших на планете? [/INST] Для начала нужно выбрать критерии величия человека. Обычно великим называют человека, который внес большой вклад в общество или сильно выделялся на фоне других в своем деле.

Например, Иосифа Бродского считают великим поэтом, а Иммануила Канта — великим философом. Александр Македонский, известный тем, что собрал в свои владения огромную империю (включавшую Македонию, Грецию, Персию, Египет), в историографии носит имя Александр Великий. Для христиан, скорее всего, самым великим человеком жившим на земле был Иисус Христос, так как он совершил множество благих деяний и совершил подвиг ради человечества. 

При этом, когда мы выдвигаем одну личность на роль великого человека, сразу же находится множество людей, не согласных с этим. Того же Иосифа Бродского, хоть он и получил престижную Нобелевскую премию, некоторые люди считают графоманом и посредственным поэтом. 

В целом, кого считать великим — это самостоятельный выбор ка

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore (F1 mean): 0.8360282778739929


In [8]:
# Print responses
print(f"Instruction: {inst_text}")
print(f"Ground Truth Response: {clean_ground_truth}")
print(f"Model Response: {clean_generated}\n")


Instruction: me quiero iniciar en el campo de la programación, ¿ que lenguaje de programación me recomiendas para empezar ?
Ground Truth Response: There are several things which you can try doing to protect yourself.
1. Increase your public awareness of your product quickly.  This could include reaching out to several influencers, large-scale advertisements , and reporters.
2. You can invest in a strong and experienced legal team.
3. You can mark up your prices so that your product has a similar cost to traditional beef sources initially.
4. Be devoted to your cause.  Don't let other companies buy you out or pay you to stop selling your product. </s> There are several strategies you can implement to lower prices over time:

Economies of scale: As you grow your business and produce more of your product, the cost per unit should decrease. You can pass these cost savings on to your customers by gradually reducing the price of your product.

Improved production processes: You can continue 